In [1]:
# Import Library
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import requests
import pandas as pd
import re
from re import search

In [2]:
# ngetes response server web nya bisa di scraping atau ngga (target: Response[200])
url = "https://www.indonesia-investments.com/id/bisnis/profil-perusahaan/item74"
page = requests.get(url)
page

<Response [200]>

In [5]:
# Setting browser
chrome_options = Options()
chrome_options.add_argument("--window-size=1920,1080")

# Ganti path dengan lokasi dari chromedriver.exe
driver = webdriver.Chrome(r"C:\Users\ddeel\Local\KerjaKerjaKerja\chromedriver.exe", chrome_options=chrome_options)
driver.get(url)
sleep(3)

<ipython-input-5-fe625fdfab4b>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\ddeel\Local\KerjaKerjaKerja\chromedriver.exe", chrome_options=chrome_options)
<ipython-input-5-fe625fdfab4b>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r"C:\Users\ddeel\Local\KerjaKerjaKerja\chromedriver.exe", chrome_options=chrome_options)


In [9]:
# Buat list untuk menampung hasil scraping
comp_names = []
descriptions = []
sectors = []
sub_sectors = []
ests = []
addresses = []
phones = []
faxs = []
emails = []
websites = []

In [10]:
# Scraping link page dari tiap company nya
links = []
for i in range(1,16): # kalau scrapnya mau dikit", range nya dikurangin aja
    url="https://www.indonesia-investments.com/id/bisnis/profil-perusahaan/item74?CompanyProfileModel_page={}&amp;ajax=yw1".format(i)
    driver.get(url)
    linknodes = driver.find_elements(By.XPATH, "//*[@class='companyname']//a")
    for node in linknodes:
        link = node.get_attribute("href")
        links.append(link)

In [12]:
# Scraping company info nya
# WARNING! will take a lot of time for running (tergantung internet speed & jumlah links)
for i in links:
    url = i
    driver.get(url)
    try:
        nama = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                By.CSS_SELECTOR, "div[class='column']>header>h1"))).text
        comp_names.append(nama)
    except:
        pass
    sleep(2)
    try:
        desc = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                By.CSS_SELECTOR, "div[class='intro medium-9']>p"))).text
        descriptions.append(desc)
    except:
        pass
        descriptions.append("")
    sleep(2)
    industry = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
                By.CSS_SELECTOR, "table[class='stacktable large-only']>tbody"))).text
    try:
        ind1 = re.split('Sektor Industri |Industry Sector', industry)[1].strip().split('\n')[0].strip()
        sectors.append(ind1)
    except:
        pass
        sectors.append("")
    sleep(2)
    try:
        ind2 = re.split('Sub Sektor |Sub Sektor Industri |Industry Sub Sector', industry)[1].strip().split('\n')[0].strip()
        sub_sectors.append(ind2)
    except:
        pass
        sub_sectors.append("")
    sleep(2)
    try:
        ind3 = re.split('Established |Didirikan', industry)[1].strip().split('\n')[0].strip()
        ests.append(ind3)
    except:
        pass
        ests.append("")
    sleep(2)
    sec = WebDriverWait(driver, 5).until(EC.presence_of_element_located((
            By.CSS_SELECTOR, "section[class='contact']>p"))).text
    kontak = re.sub('\n', ' ', sec)
    try:
        alamat = re.split('Phone: |Telpon: |Telepon: |Fax: ',kontak)[0]
        addresses.append(alamat)
    except:
        pass
        addresses.append("")
    sleep(1)
    if search('Phone|Telpon|Telepon', kontak):
        telp = re.split('Phone: |Telpon: |Telepon: |Fax: |Faks: |Email:|E-mail:|www',kontak)[1]
        phones.append(telp)
    else:
        phones.append("")
    if search('Fax|Faks|Faksimile', kontak):
        fak = re.split('Faks: |Fax: |Email: |www', kontak)[1]
        faxs.append(fak)
    else:
        faxs.append("")
    if search('Email|E-mail', kontak):
        em = re.split('Email: |www', kontak)[1]
        emails.append(em)
    else:
        emails.append("")
    if search('www', kontak):
        website = 'www' + kontak.split('www')[1]
        websites.append(website)
    else:
        websites.append("")
    sleep(3)

In [13]:
# input hasil scraping ke dataframe pandas
df = pd.DataFrame()
df['Company Name'] = comp_names
df['Description'] = descriptions
df['Industry'] = sectors
df['Specialties'] = sub_sectors
df['Established'] = ests
df['Address'] = addresses
df['Phone'] = phones
df['Fax'] = faxs
df['E-mail'] = emails
df['Website'] = websites

In [14]:
df.head()

,Company Name,Description,Industry,Specialties,Established,Address,Phone,Fax,E-mail,Website
0,ABM Investama,PT ABM Investama Tbk is an Indonesia-based inv...,"Trade, Services & Investment",Investment Company,1 June 2006,"TMT Building 1, 18th Floor Jalan Cilandak KKO ...",+62 21 299 76767,+62 21 299 76768,corporate-secretary@abm-investama.co.id,www.abm-investama.com
1,ACE Hardware Indonesia,PT ACE Hardware Indonesia Tbk adalah sebuah pe...,"Perdagangan, Jasa & Investasi",Industri Perdagangan Ritel,3 Februari 1995,Kawan Lama Building Lantai 5 Jalan Puri Kencan...,+62 21 582 2222,+62 21 582 4022,helen_tanzil@acehardware.co.id,www.acehardware.co.id
2,Acset Indonusa,PT Acset Indonusa Tbk is an Indonesia-based co...,"Property, Real Estate & Building Construction",Building Construction,10 January 1995,"Acset Building, Jalan Majapahit No. 26 South P...",+62 21 351 1961,+62 21 3441 413,corporate.secretary@acset.co,www.acset.co
3,Adaro Energy,PT Adaro Energy Tbk adalah perusahaan pertamba...,Pertambangan,Industri Batu Bara,28 Juli 2004,"Menara Karya, 23rd Floor Jln H.R. Rasuna Said ...",+62 21 2553 3000,+62 21 5211 266,mahardika.putranto@ptadaro.com,www.adaro.com
4,Adhi Karya,Adhi Karya adalah sebuah perusahaan Indonesia ...,"Properti, Real Estate & Konstruksi",,11 Maret 1960,Jalan Raya Pasar Minggu Km. 18 Jakarta 12510,+62 21 797 5312 Faks: +62 21 797 5311 Email: e...,+62 21 797 5311,e.adhinet@adhi.co.id,www.adhi.co.id


In [15]:
# Export the scrap result into xlsx file
df.to_excel('result.xlsx')

C:\Users\ddeel\anaconda3\lib\site-packages\pandas\compat\_optional.py:123: UserWarning: Pandas requires version '1.0.2' or newer of 'xlsxwriter' (version '0.7.3' currently installed).
  warnings.warn(msg, UserWarning)
